In [1]:
import pandas as pd
import geopandas as gpd

In [2]:
## import custom functions

from pathlib import Path
import sys

# Create path to custom functions
funcs_path = Path.cwd().parents[1]
sys.path.insert(1, str(funcs_path))

from funcs import paths

In [3]:
path_to_Dublin_SA_energy_using_archetypes = paths.get_path_to_data('Dublin_SA_energy_using_allDublin_sample.pkl') 

path_to_Dublin_SA_energy_using_archetypes

'C:\\Users\\RowanM\\Documents\\dublin_energy_masterplan\\Residential\\data\\models\\Dublin_SA_energy_using_allDublin_sample.pkl'

In [4]:
Dublin_SA_energy_using_archetypes = pd.read_pickle(path_to_Dublin_SA_energy_using_archetypes,compression='xz')

Dublin_SA_energy_using_archetypes.head()

,Small Area,HW,SH,TotalHH
0,267001001,197070.857401,506420.632028,65.0
1,267001002,269835.481672,693406.711547,89.0
2,267001003,294090.356429,755735.404719,97.0
3,267001004,315313.371842,810273.011245,104.0
4,267001005,339568.246599,872601.704418,112.0


In [5]:
path_to_Dublin_2011_shp = paths.get_path_to_data('Dublin2011_with_Pcodes') 

path_to_Dublin_2011_shp

'C:\\Users\\RowanM\\Documents\\dublin_energy_masterplan\\Residential\\data\\inputs\\GIS\\Dublin2011_with_Pcodes\\Dublin2011_with_Pcodes.shp'

In [6]:
gdf = gpd.read_file(path_to_Dublin_2011_shp)[["SMALL_AREA","DataFram_1","geometry"]]
gdf.columns = ["SA", "Postcode", "geometry"]
gdf.set_index("SA",inplace=True)

gdf

,Postcode,geometry
SA,,
267123023,Co. Dublin,"POLYGON ((325504.506 259932.025, 325502.052 25..."
268073009,Dublin 13,"POLYGON ((322950.711 239928.508, 322991.821 23..."
268064003,Dublin 11,"POLYGON ((312555.357 239682.147, 312560.637 23..."
267028079,Dublin 15,"POLYGON ((303688.553 239832.927, 303618.871 23..."
267028076,Dublin 15,"POLYGON ((304415.356 239637.446, 304342.628 23..."
...,...,...
267078013/02,Co. Dublin,"POLYGON ((319395.811 226949.205, 319377.643 22..."
267078013/01,Co. Dublin,"POLYGON ((319067.441 226839.903, 319004.981 22..."
267078011/02,Co. Dublin,"POLYGON ((319028.251 226664.189, 319045.416 22..."


In [8]:
# Merge dataframes gdf and df_energy
merged = gdf.merge(Dublin_SA_energy_using_archetypes, left_on = 'SA', right_on = 'Small Area', how="left")

merged.rename(columns={"Small Area":"SA"},inplace=True)

merged

,Postcode,geometry,SA,HW,SH,TotalHH
0,Co. Dublin,"POLYGON ((325504.506 259932.025, 325502.052 25...",267123023,297122.215774,7.635265e+05,98.0
1,Dublin 13,"POLYGON ((322950.711 239928.508, 322991.821 23...",268073009,418396.589559,1.075170e+06,138.0
2,Dublin 11,"POLYGON ((312555.357 239682.147, 312560.637 23...",268064003,236485.028881,6.077048e+05,78.0
3,Dublin 15,"POLYGON ((303688.553 239832.927, 303618.871 23...",267028079,312281.512497,8.024819e+05,103.0
4,Dublin 15,"POLYGON ((304415.356 239637.446, 304342.628 23...",267028076,409301.011525,1.051797e+06,135.0
...,...,...,...,...,...,...
4801,Co. Dublin,"POLYGON ((319395.811 226949.205, 319377.643 22...",267078013/02,500256.791864,1.285529e+06,165.0
4802,Co. Dublin,"POLYGON ((319067.441 226839.903, 319004.981 22...",267078013/01,100051.358373,2.571059e+05,33.0
4803,Co. Dublin,"POLYGON ((319028.251 226664.189, 319045.416 22...",267078011/02,263771.762983,6.778245e+05,87.0
4804,Dublin 15,"POLYGON ((303721.883 239179.308, 303874.615 23...",267028004/01,357759.402667,9.193482e+05,118.0


Source code (modified): 

Plotly Choropleth Map Tutorial by Shivangi Patel:
- https://towardsdatascience.com/a-complete-guide-to-an-interactive-geographical-map-using-python-f4c5197e23e0

In [9]:
## import modules

import json
import os

# -----------------------

# Bokeh map drawing
from bokeh.io import output_notebook, show, output_file
from bokeh.plotting import figure
from bokeh.models import GeoJSONDataSource, LinearColorMapper, ColorBar
from bokeh.palettes import brewer

# Bokeh map interactivity
from bokeh.io import curdoc, output_notebook
from bokeh.models import Slider, HoverTool
from bokeh.layouts import widgetbox, row, column

# Source: https://towardsdatascience.com/a-complete-guide-to-an-interactive-geographical-map-using-python-f4c5197e23e0
# -----------------------

# Bokeh dropdown menu
from bokeh.io import output_file, show
from bokeh.models.widgets import Dropdown

# Source: https://docs.bokeh.org/en/latest/docs/user_guide/interaction/widgets.html
# -----------------------

In [10]:
#Read data to json.
merged_json = json.loads(merged.to_json())

#Convert to String like object.
json_data = json.dumps(merged_json)

In [11]:
# Input GeoJSON source that contains features for plotting.
geosource = GeoJSONDataSource(geojson=json_data)

# Define a sequential multi-hue color palette.
palette = brewer["YlGnBu"][8]

# Reverse color order so that dark blue is highest obesity.
palette = palette[::-1]

# Instantiate LinearColorMapper that linearly maps numbers in a range, into a sequence of colors. Input nan_color.
min_value = merged["SH"].min()
max_value = merged["SH"].max()
color_mapper = LinearColorMapper(
    palette=palette, low=min_value, high=max_value, nan_color="#d9d9d9"
)

# Add hover tool
hover = HoverTool(tooltips=[("Small Area", "@SA"), ("SH", "@SH"),])

# Create color bar.
color_bar = ColorBar(
    color_mapper=color_mapper,
    label_standoff=8,
    width=500,
    height=20,
    border_line_color=None,
    location=(0, 0),
    orientation="horizontal",
)

# Create figure object.
p = figure(
    title="Dublin SAs 2016",
    plot_height=950,
    plot_width=950,
    toolbar_location=None,
    tools=[hover],
)
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None

# Add patch renderer to figure.
p.patches(
    "xs",
    "ys",
    source=geosource,
    fill_color={"field": "SH", "transform": color_mapper},
    line_color="black",
    line_width=0.25,
    fill_alpha=1,
)

# Specify figure layout.
p.add_layout(color_bar, "below")

# Must define dropdown before function to call
# Source: https://stackoverflow.com/questions/39100095/how-to-capture-value-of-dropdown-widget-in-bokeh-python
menu = [
    ("Space Heat Demand [MWh/year]", "SH"),
    ("Hot Water Demand [MWh/year]", "HW"),
    ("Total Number of Households", "TotalHH"),
]
dropdown = Dropdown(
    label="Click here to change the data being mapped",
    button_type="warning",
    menu=menu,
    orientation="vertical",
)


def return_full_column_name(select):

    if select == "SH":
        return "Space Heat Demand [MWh/year]"

    if select == "HW":
        return "Hot Water Demand [MWh/year]"

    if select == "TotalHH":
        return "Total Number of Households"


# Define the callback function: update_plot
def update_plot(attr, old, new):

    select = dropdown.value

    min_value = merged[select].min()
    max_value = merged[select].max()
    color_mapper = LinearColorMapper(
        palette=palette, low=min_value, high=max_value, nan_color="#d9d9d9"
    )

    # Add hover tool
    hover = HoverTool(
        tooltips=[
            ("Small Area", "@SA"),
            (return_full_column_name(select), "@" + select),
        ]
    )
    p.tools = [hover]
    # Add patch renderer to figure.
    p.patches(
        "xs",
        "ys",
        source=geosource,
        fill_color={"field": select, "transform": color_mapper},
        line_color="black",
        line_width=0.25,
        fill_alpha=1,
    )
    return None


dropdown.on_change("value", update_plot)

# Make a column layout of widgetbox(slider) and plot, and add it to the current document
layout = row(p, widgetbox(dropdown))
curdoc().add_root(layout)

# Display plot inline in Jupyter notebook
output_notebook()
# output_file("Dublin_SAs.html")

# Display plot
show(layout)

Loading BokehJS ...

You are generating standalone HTML/JS output, but trying to use real Python
callbacks (i.e. with on_change or on_event). This combination cannot work.

Only JavaScript callbacks may be used with standalone output. For more
information on JavaScript callbacks with Bokeh, see:

    https://docs.bokeh.org/en/latest/docs/user_guide/interaction/callbacks.html

Alternatively, to use real Python callbacks, a Bokeh server application may
be used. For more information on building and running Bokeh applications, see:

    https://docs.bokeh.org/en/latest/docs/user_guide/server.html



To update the plot when change the slider need to setup a local Bokeh server!

```
bokeh serve --show filename.ipynb
```

Bokeh widget examples (including dropdown):
- https://docs.bokeh.org/en/latest/docs/user_guide/interaction/widgets.html
- https://towardsdatascience.com/data-visualization-with-bokeh-in-python-part-ii-interactions-a4cf994e2512
    